In [110]:
# Import program libraries and the data set
import numpy as np
import scipy.stats as sts
import scipy.special as spc
import scipy.optimize as opt
import scipy.integrate as intg
import matplotlib.pyplot as ply
import pandas as pd
from pandas import Series, DataFrame

data = np.loadtxt('MacroSeries.txt', delimiter = ',')

# Make the data set time-series
df = pd.DataFrame(data, columns = ['c', 'k', 'w', 'r'], index = np.arange(1,101))

In [111]:
# Assign variables to each column of the data set
c = df.loc[:,'c']
k = df.loc[:,'k']
w = df.loc[:,'w']
r = df.loc[:,'r']

In [112]:
#(a)
# Define and solve Equation (3) for z_t
def eqn3(w,k,alpha):
    z = np.log(w)-np.log(1-alpha)-alpha*np.log(k)
    return z

In [113]:
# Compute epsilon from Equation (5)
def eps(z,rho,mu):
    z_lag = np.append(mu,z[:-1])
    eps = z - rho* z_lag - (1-rho) * mu
    return eps

In [114]:
# Define pdf of the normal distribution
def normal_pdf(z, rho, mu, sigma):
    xvals = eps(z,rho,mu)
    pdf_vals = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-(xvals**2)/(2*sigma**2))
    return pdf_vals

# Define log likelihood function for the normal distribution
def log_lik_norm(z, rho, mu, sigma):
    pdf_vals = normal_pdf(z, rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val

# Define the criterion function
def crit(params, *args):
    alpha, rho, mu, sigma = params
    w, k = args
    z = eqn3(w,k,alpha)
    log_lik_val = log_lik_norm(z, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

# Set initial values and bounds
alpha_0 = 0.2
rho_0 = 0.7
mu_0 = 10
sigma_0 = 0.3
params_init = np.array([alpha_0, rho_0, mu_0, sigma_0])
param_bounds = ((1e-10, 1-(1e-10)),(-1+(1e-10), 1-(1e-10)), (1e-10, None),(1e-10, None))

# ML Estimation
mle_args = (w, k)
results = opt.minimize(crit, params_init, args=(mle_args), bounds = param_bounds, method = 'L-BFGS-B')
alpha_mle, rho_mle, mu_mle, sigma_mle = results.x
print('alpha_mle', alpha_mle, 'rho_mle', rho_mle, 'mu_mle', mu_mle, 'sigma_mle', sigma_mle)

alpha_mle 0.317361326029 rho_mle 0.775089983587 mu_mle 10.0813859106 sigma_mle 1.02121646634


/Users/nobuyukifuruta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/Users/nobuyukifuruta/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:628: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]


In [117]:
# Commpute the inverse hessian variance-covariance matrix of the estimates
vcv_mle = results.hess_inv*np.identity(4)
print ('VCV(MLE)=', vcv_mle)

VCV(MLE)= [[ 0.63630516 -0.51864509 -0.07810611 -0.53205902]
 [-0.51864509  0.48491465 -0.00471448  0.30049074]
 [-0.07810611 -0.00471448  1.00860882  0.03837371]
 [-0.53205902  0.30049074  0.03837371  0.76698956]]


In [120]:
#(b)
# Define and solve Equation (4) for z_t
def eqn4(r,k,alpha):
    z = np.log(r)-np.log(alpha)-(alpha-1)*np.log(k)
    return z

# Compute epsilon from Equation (5)
def eps(z,rho,mu):
    z_lag = np.append(mu,z[:-1])
    eps = z - rho* z_lag - (1-rho) * mu
    return eps

In [127]:
# Define pdf of the normal distribution
def normal_pdf(z, rho, mu, sigma):
    xvals = eps(z,rho,mu)
    pdf_vals = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-(xvals**2)/(2*sigma**2))
    return pdf_vals

# Define log likelihood function for the normal distribution
def log_lik_norm(z, rho, mu, sigma):
    pdf_vals = normal_pdf(z, rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val

# Define the criterion function
def crit(params, *args):
    alpha, rho, mu, sigma = params
    r, k = args
    z = eqn4(r,k,alpha)
    log_lik_val = log_lik_norm(z, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

# Set initial values and bounds
alpha_0 = 0.2
rho_0 = 0.3
mu_0 = 15
sigma_0 = 0.3
params_init = np.array([alpha_0, rho_0, mu_0, sigma_0])
param_bounds = ((1e-10, 1-(1e-10)),(-1+(1e-10), 1-(1e-10)), (1e-10, None),(1e-10, None))

# ML Estimation
mle_args = (r, k)
results = opt.minimize(crit, params_init, args=(mle_args), bounds = param_bounds, method = 'L-BFGS-B')
alpha_mle2, rho_mle2, mu_mle2, sigma_mle2 = results.x
print('alpha_mle2', alpha_mle2, 'rho_mle2', rho_mle2, 'mu_mle2', mu_mle2, 'sigma_mle2', sigma_mle2)

alpha_mle2 0.162407213636 rho_mle2 0.304021206717 mu_mle2 14.9932771952 sigma_mle2 0.531144188169


/Users/nobuyukifuruta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/Users/nobuyukifuruta/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:628: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]


In [128]:
# Commpute the inverse hessian variance-covariance matrix of the estimates
vcv_mle = results.hess_inv*np.identity(4)
print ('VCV(MLE)=', vcv_mle)

VCV(MLE)= [[  4.11843417e-02   1.20300508e-02  -1.92974843e-02  -1.20024680e+00]
 [  1.20300508e-02   1.01068891e+00   2.69574945e-02  -1.29683419e+00]
 [ -1.92974843e-02   2.69574945e-02   1.01421072e+00  -7.70272125e-01]
 [ -1.20024680e+00  -1.29683419e+00  -7.70272125e-01   3.75640494e+01]]


In [130]:
#(c)
# Solve Equation (4) for z* given r=1, k=7500000
z_star = -np.log(alpha_mle)-(alpha_mle -1)*np.log(7500000)

In [137]:
# Compute the probability that r<=1 given MLE parametes, r=1, and k=7500000
def normal_pdf(z,rho,mu,sigma):
    pdf_vals = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-((z-10*rho+(1-rho)*mu)**2)/(2*sigma**2))
    return pdf_vals
prob = 1- intg.quad(lambda z: normal_pdf(z,rho_mle,mu_mle,sigma_mle), -np.inf, z_star)[0]
print('probability=', prob)

probability= 1.0
